In [12]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.1.0


In [13]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to C:\Users\rexdi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteOSV2N3\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\rexdi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteOSV2N3\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\rexdi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteOSV2N3\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\rexdi\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [14]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [15]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [16]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? b for me this is a story that starts with some funny jokes regarding <OOV> <OOV> when he is travelling with a <OOV> and when he is sitting in business <OOV> the problem is that when you have been watching this movie for an hour you will see the same fantasies funny situations again and again and again it is to predictable it is more done as a tv story where you can go away and come back without missing anything br br i like felix <OOV> as frank but that is not enough even when it is a comedy it has to have more variations and some kind of message to it's audience br br
b"For me this is a story that starts with some funny jokes regarding Franks fanatasies when he is travelling with a staircase and when he is sitting in business meetings... The problem is that when you have been watching this movie for an hour you will see the same fantasies/funny situations again and again and again. It is to predictable. It is more done as a TV story where you can go away and come back without 

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [18]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - ETA: 17:05 - loss: 0.6906 - accuracy: 0.718 - ETA: 2:56 - loss: 0.6937 - accuracy: 0.557 - ETA: 1:24 - loss: 0.6937 - accuracy: 0.50 - ETA: 56s - loss: 0.6934 - accuracy: 0.5141 - ETA: 42s - loss: 0.6935 - accuracy: 0.496 - ETA: 34s - loss: 0.6932 - accuracy: 0.502 - ETA: 29s - loss: 0.6932 - accuracy: 0.500 - ETA: 25s - loss: 0.6926 - accuracy: 0.513 - ETA: 23s - loss: 0.6921 - accuracy: 0.520 - ETA: 21s - loss: 0.6913 - accuracy: 0.528 - ETA: 19s - loss: 0.6913 - accuracy: 0.526 - ETA: 17s - loss: 0.6916 - accuracy: 0.522 - ETA: 16s - loss: 0.6913 - accuracy: 0.524 - ETA: 15s - loss: 0.6910 - accuracy: 0.530 - ETA: 14s - loss: 0.6904 - accuracy: 0.536 - ETA: 13s - loss: 0.6904 - accuracy: 0.538 - ETA: 13s - loss: 0.6901 - accuracy: 0.538 - ETA: 12s - loss: 0.6892 - accuracy: 0.544 - ETA: 12s - loss: 0.6890 - accuracy: 0.550 - ETA: 11s - loss: 0.6884 - accuracy: 0.555 - ETA: 11s

25000/25000 [==============================] - ETA: 7s - loss: 0.2216 - accuracy: 0.90 - ETA: 6s - loss: 0.1648 - accuracy: 0.95 - ETA: 6s - loss: 0.2223 - accuracy: 0.91 - ETA: 6s - loss: 0.2233 - accuracy: 0.92 - ETA: 5s - loss: 0.2234 - accuracy: 0.91 - ETA: 5s - loss: 0.2268 - accuracy: 0.91 - ETA: 5s - loss: 0.2262 - accuracy: 0.91 - ETA: 5s - loss: 0.2242 - accuracy: 0.91 - ETA: 5s - loss: 0.2252 - accuracy: 0.92 - ETA: 5s - loss: 0.2269 - accuracy: 0.92 - ETA: 5s - loss: 0.2274 - accuracy: 0.92 - ETA: 5s - loss: 0.2254 - accuracy: 0.92 - ETA: 5s - loss: 0.2258 - accuracy: 0.92 - ETA: 5s - loss: 0.2282 - accuracy: 0.92 - ETA: 5s - loss: 0.2284 - accuracy: 0.92 - ETA: 5s - loss: 0.2303 - accuracy: 0.91 - ETA: 5s - loss: 0.2336 - accuracy: 0.91 - ETA: 5s - loss: 0.2331 - accuracy: 0.91 - ETA: 5s - loss: 0.2318 - accuracy: 0.91 - ETA: 4s - loss: 0.2370 - accuracy: 0.91 - ETA: 4s - loss: 0.2351 - accuracy: 0.91 - ETA: 4s - loss: 0.2372 - accuracy: 0.91 - ETA: 4s - loss: 0.2388 - accu

25000/25000 [==============================] - ETA: 7s - loss: 0.0746 - accuracy: 1.00 - ETA: 7s - loss: 0.0898 - accuracy: 0.98 - ETA: 6s - loss: 0.0923 - accuracy: 0.98 - ETA: 6s - loss: 0.0935 - accuracy: 0.98 - ETA: 6s - loss: 0.1009 - accuracy: 0.97 - ETA: 6s - loss: 0.0982 - accuracy: 0.98 - ETA: 6s - loss: 0.0962 - accuracy: 0.98 - ETA: 6s - loss: 0.1057 - accuracy: 0.97 - ETA: 6s - loss: 0.1056 - accuracy: 0.97 - ETA: 5s - loss: 0.1039 - accuracy: 0.97 - ETA: 5s - loss: 0.1040 - accuracy: 0.97 - ETA: 5s - loss: 0.1020 - accuracy: 0.97 - ETA: 5s - loss: 0.1019 - accuracy: 0.97 - ETA: 5s - loss: 0.1007 - accuracy: 0.97 - ETA: 5s - loss: 0.1000 - accuracy: 0.97 - ETA: 5s - loss: 0.0995 - accuracy: 0.97 - ETA: 5s - loss: 0.0987 - accuracy: 0.97 - ETA: 5s - loss: 0.0984 - accuracy: 0.97 - ETA: 5s - loss: 0.0980 - accuracy: 0.97 - ETA: 5s - loss: 0.0995 - accuracy: 0.97 - ETA: 5s - loss: 0.0996 - accuracy: 0.97 - ETA: 4s - loss: 0.0988 - accuracy: 0.97 - ETA: 4s - loss: 0.1013 - accu

25000/25000 [==============================] - ETA: 9s - loss: 0.0248 - accuracy: 1.00 - ETA: 6s - loss: 0.0225 - accuracy: 1.00 - ETA: 6s - loss: 0.0240 - accuracy: 1.00 - ETA: 6s - loss: 0.0238 - accuracy: 1.00 - ETA: 5s - loss: 0.0256 - accuracy: 0.99 - ETA: 5s - loss: 0.0265 - accuracy: 0.99 - ETA: 5s - loss: 0.0280 - accuracy: 0.99 - ETA: 5s - loss: 0.0276 - accuracy: 0.99 - ETA: 5s - loss: 0.0267 - accuracy: 0.99 - ETA: 5s - loss: 0.0281 - accuracy: 0.99 - ETA: 5s - loss: 0.0270 - accuracy: 0.99 - ETA: 5s - loss: 0.0270 - accuracy: 0.99 - ETA: 5s - loss: 0.0263 - accuracy: 0.99 - ETA: 5s - loss: 0.0256 - accuracy: 0.99 - ETA: 5s - loss: 0.0257 - accuracy: 0.99 - ETA: 5s - loss: 0.0253 - accuracy: 0.99 - ETA: 5s - loss: 0.0249 - accuracy: 0.99 - ETA: 5s - loss: 0.0245 - accuracy: 0.99 - ETA: 5s - loss: 0.0246 - accuracy: 0.99 - ETA: 5s - loss: 0.0254 - accuracy: 0.99 - ETA: 5s - loss: 0.0260 - accuracy: 0.99 - ETA: 5s - loss: 0.0256 - accuracy: 0.99 - ETA: 5s - loss: 0.0251 - accu

25000/25000 [==============================] - ETA: 8s - loss: 0.0079 - accuracy: 1.00 - ETA: 6s - loss: 0.0066 - accuracy: 1.00 - ETA: 6s - loss: 0.0063 - accuracy: 1.00 - ETA: 5s - loss: 0.0068 - accuracy: 1.00 - ETA: 5s - loss: 0.0102 - accuracy: 0.99 - ETA: 5s - loss: 0.0100 - accuracy: 0.99 - ETA: 5s - loss: 0.0094 - accuracy: 0.99 - ETA: 5s - loss: 0.0103 - accuracy: 0.99 - ETA: 5s - loss: 0.0100 - accuracy: 0.99 - ETA: 5s - loss: 0.0097 - accuracy: 0.99 - ETA: 5s - loss: 0.0095 - accuracy: 0.99 - ETA: 5s - loss: 0.0090 - accuracy: 0.99 - ETA: 5s - loss: 0.0087 - accuracy: 0.99 - ETA: 5s - loss: 0.0086 - accuracy: 0.99 - ETA: 5s - loss: 0.0083 - accuracy: 0.99 - ETA: 5s - loss: 0.0081 - accuracy: 0.99 - ETA: 5s - loss: 0.0080 - accuracy: 0.99 - ETA: 5s - loss: 0.0079 - accuracy: 0.99 - ETA: 4s - loss: 0.0078 - accuracy: 0.99 - ETA: 4s - loss: 0.0077 - accuracy: 0.99 - ETA: 4s - loss: 0.0076 - accuracy: 0.99 - ETA: 4s - loss: 0.0075 - accuracy: 0.99 - ETA: 4s - loss: 0.0074 - accu

...

25000/25000 [==============================] - ETA: 7s - loss: 5.1794e-04 - accuracy: 1.00 - ETA: 6s - loss: 5.3668e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.3726e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.3413e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.5765e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.5246e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.5095e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.5260e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.4990e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.6913e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.6239e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.5551e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.4439e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.4826e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.5008e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.4395e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.4200e-04 - accuracy: 1.00 - ETA: 5s - loss: 5.4182e-04 - accuracy: 1.00 - ETA: 4s - loss: 5.4061e-04 - accuracy: 1.00 - ETA: 4s - loss: 5.3731e-04 - accuracy: 1.00 - ETA: 4s - loss: 5.3500e-04 - accu

24192/25000 [============================>.] - ETA: 7s - loss: 1.8675e-04 - accuracy: 1.00 - ETA: 6s - loss: 2.7131e-04 - accuracy: 1.00 - ETA: 6s - loss: 2.7258e-04 - accuracy: 1.00 - ETA: 6s - loss: 2.8274e-04 - accuracy: 1.00 - ETA: 6s - loss: 3.1150e-04 - accuracy: 1.00 - ETA: 6s - loss: 3.1379e-04 - accuracy: 1.00 - ETA: 6s - loss: 3.1053e-04 - accuracy: 1.00 - ETA: 6s - loss: 3.0403e-04 - accuracy: 1.00 - ETA: 6s - loss: 2.9904e-04 - accuracy: 1.00 - ETA: 6s - loss: 3.0033e-04 - accuracy: 1.00 - ETA: 5s - loss: 3.0720e-04 - accuracy: 1.00 - ETA: 5s - loss: 3.0879e-04 - accuracy: 1.00 - ETA: 5s - loss: 3.0408e-04 - accuracy: 1.00 - ETA: 5s - loss: 2.9985e-04 - accuracy: 1.00 - ETA: 5s - loss: 2.9934e-04 - accuracy: 1.00 - ETA: 5s - loss: 3.0331e-04 - accuracy: 1.00 - ETA: 5s - loss: 3.0133e-04 - accuracy: 1.00 - ETA: 5s - loss: 3.0083e-04 - accuracy: 1.00 - ETA: 5s - loss: 2.9862e-04 - accuracy: 1.00 - ETA: 5s - loss: 2.9578e-04 - accuracy: 1.00 - ETA: 5s - loss: 2.9495e-04 - accu

25000/25000 [==============================] - ETA: 0s - loss: 2.7673e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7636e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7629e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7612e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7621e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7605e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7582e-04 - accuracy: 1.00 - ETA: 0s - loss: 2.7550e-04 - accuracy: 1.00 - 16s 645us/sample - loss: 2.7541e-04 - accuracy: 1.0000 - val_loss: 0.7690 - val_accuracy: 0.8293
Epoch 10/10


18560/25000 [=====================>........] - ETA: 16s - loss: 1.3129e-04 - accuracy: 1.000 - ETA: 15s - loss: 1.6085e-04 - accuracy: 1.000 - ETA: 14s - loss: 1.6555e-04 - accuracy: 1.000 - ETA: 15s - loss: 1.7669e-04 - accuracy: 1.000 - ETA: 14s - loss: 1.7865e-04 - accuracy: 1.000 - ETA: 14s - loss: 1.7526e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7684e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7937e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7837e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7974e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7863e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7976e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.8063e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7748e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.8057e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7798e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.7561e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.8488e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.8498e-04 - accuracy: 1.000 - ETA: 13s - loss: 1.8832e-04 - accuracy: 1

25000/25000 [==============================] - ETA: 3s - loss: 1.7235e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.7199e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.7189e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.7185e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.7202e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.7204e-04 - accuracy: 1.00 - ETA: 3s - loss: 1.7193e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7177e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7160e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7129e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7113e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7145e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7124e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7098e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7095e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7092e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7074e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7064e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7052e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7026e-04 - accuracy: 1.00 - ETA: 2s - loss: 1.7013e-04 - accu

In [19]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [20]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [21]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [22]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1429], [969], [4], [1537], [1537], [4749], [], [790], [2012], [11], [2920], [2188], [], [790], [2012], [11], [579], [], [11], [579], [], [4], [1783], [4], [4512], [11], [2920], [1277], [], [], [2012], [1003], [2920], [969], [579], [790], []]
